In [1]:
import pandas as pd 
import glob
import cv2 as cv
import random
import os

import matplotlib.pyplot as plt
import numpy as np
import random
from PIL import Image
import PIL.ImageOps    

import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
import torchvision.utils
import torch
from torch.autograd import Variable
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
from sklearn.metrics import f1_score, accuracy_score


from torch.cuda.amp import autocast
from torch.cuda.amp import GradScaler
from tqdm.auto import tqdm
import timm
import math
from sklearn.model_selection import train_test_split
import pytorch_model_summary



c:\Users\EHmin\anaconda3\envs\EHmin\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Make DataFrame

## columns: review_img_path, product_img_path, label
---

In [2]:
all_product_img_list = glob.glob('./masked_data/product_img/*')
all_product_img_list = sorted(all_product_img_list)
print(all_product_img_list)

['./masked_data/product_img\\0.jpg', './masked_data/product_img\\1.jpg', './masked_data/product_img\\10.jpg', './masked_data/product_img\\100.jpg', './masked_data/product_img\\12.jpg', './masked_data/product_img\\13.jpg', './masked_data/product_img\\14.jpg', './masked_data/product_img\\15.jpg', './masked_data/product_img\\20.jpg', './masked_data/product_img\\25.jpg', './masked_data/product_img\\28.jpg', './masked_data/product_img\\30.jpg', './masked_data/product_img\\33.jpg', './masked_data/product_img\\34.jpg', './masked_data/product_img\\35.jpg', './masked_data/product_img\\36.jpg', './masked_data/product_img\\37.jpg', './masked_data/product_img\\39.jpg', './masked_data/product_img\\4.jpg', './masked_data/product_img\\40.jpg', './masked_data/product_img\\41.jpg', './masked_data/product_img\\42.jpg', './masked_data/product_img\\43.jpg', './masked_data/product_img\\45.jpg', './masked_data/product_img\\46.jpg', './masked_data/product_img\\48.jpg', './masked_data/product_img\\49.jpg', '.

In [3]:
review_img_path = []
product_img_path = []
label = []
for product_img in all_product_img_list:
    glob_img_list = []
    str = product_img.split("\\")[1] # for window
    # str = product_img.split("/")[3] # for linux
    
    str = str.split(".")[0]
    glob_img_list = glob.glob('./masked_data/review_img/'+str +'_review_img/O/*.jpg')
    review_img_path = review_img_path + glob_img_list
    product_img_path = product_img_path + [product_img  for i in range(len(glob_img_list))]

In [4]:
print(len(product_img_path))
print(len(review_img_path))

18749
18749


In [5]:
for i in range(len(review_img_path)):
    random_number = random.randint(0, 1)
    if random_number == 0:
        label.append(0)
    else:
        review_num = review_img_path[i].split('\\')[1] # for Window 
        # review_num = review_img_path[i].split('/')[3] # for Linux
        
        review_num = review_num.split("_")[0]
        random_number = random.randint(0, len(all_product_img_list) - 1)
        while review_num == random_number:
            random_number = random.randint(0, len(all_product_img_list) - 1)
        product_img_path[i] = all_product_img_list[random_number]
        label.append(1)

In [6]:
df = pd.DataFrame(columns=['review_img_path','product_img_path', 'label'])
df['review_img_path'] = review_img_path
df['product_img_path'] = product_img_path
df['label'] = label

df

,review_img_path,product_img_path,label
0,./masked_data/review_img/0_review_img/O\1.jpg,./masked_data/product_img\0.jpg,0
1,./masked_data/review_img/0_review_img/O\107.jpg,./masked_data/product_img\0.jpg,0
2,./masked_data/review_img/0_review_img/O\109.jpg,./masked_data/product_img\0.jpg,0
3,./masked_data/review_img/0_review_img/O\111.jpg,./masked_data/product_img\0.jpg,0
4,./masked_data/review_img/0_review_img/O\119.jpg,./masked_data/product_img\0.jpg,0
...,...,...,...
18744,./masked_data/review_img/98_review_img/O\93.jpg,./masked_data/product_img\79.jpg,1
18745,./masked_data/review_img/98_review_img/O\95.jpg,./masked_data/product_img\98.jpg,0
18746,./masked_data/review_img/98_review_img/O\96.jpg,./masked_data/product_img\61.jpg,1
18747,./masked_data/review_img/98_review_img/O\97.jpg,./masked_data/product_img\98.jpg,0


---

# Siames Net

In [7]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [8]:
CFG = {
    'IMG_SIZE':224,
    'EPOCHS':100,
    'LEARNING_RATE':3e-4,
    # 'LEARNING_RATE':10,
    'BATCH_SIZE':8,
    'SEED':41
}

In [9]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = 'a'
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

In [10]:
train, val, _, _ = train_test_split(df, df['label'], test_size=0.3, stratify=df['label'], random_state=CFG['SEED'])

In [11]:
class SiameseNetworkDataset(Dataset):
    def __init__(self,review_img_path,product_img_path,label,transform=None):
        self.review_img_path = review_img_path
        self.product_img_path = product_img_path
        self.label = label
        self.transform = transform
        
    def __getitem__(self,index):
        review_img = cv.imread(self.review_img_path[index])
        product_img = cv.imread(self.product_img_path[index])
        review_img = cv.resize(review_img, (CFG['IMG_SIZE'], CFG['IMG_SIZE']))
        product_img = cv.resize(product_img, (CFG['IMG_SIZE'], CFG['IMG_SIZE']))

        if self.transform is not None:
            review_img  = self.transform(image=review_img)['image']
            product_img  = self.transform(image=product_img)['image']
    
        return review_img, product_img, self.label[index]
    
    def __len__(self):
        return len(self.review_img_path)

In [12]:
train_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            ToTensorV2()
                            ])

In [13]:
train_dataset = SiameseNetworkDataset(train["review_img_path"].values, train["product_img_path"].values, train["label"].values, train_transform)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

val_dataset = SiameseNetworkDataset(val["review_img_path"].values, val["product_img_path"].values, val["label"].values, train_transform)
val_loader = DataLoader(val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [14]:
avail_pretrained_models = timm.list_models('*VGG*',pretrained=True)
print(avail_pretrained_models)

['repvgg_a2', 'repvgg_b0', 'repvgg_b1', 'repvgg_b1g4', 'repvgg_b2', 'repvgg_b2g4', 'repvgg_b3', 'repvgg_b3g4', 'vgg11', 'vgg11_bn', 'vgg13', 'vgg13_bn', 'vgg16', 'vgg16_bn', 'vgg19', 'vgg19_bn']


In [15]:
class BaseModel(nn.Module):
    def __init__(self):
        super(BaseModel, self).__init__()
        self.backbone = timm.create_model('efficientnet_b0', pretrained=False)
        self.classifier = nn.Linear(1000, 50)
        self.dropout = nn.Dropout(0.1)
        self.ReLU = nn.ReLU(inplace=False)
        
    def forward(self, x, y):
        x = self.backbone(x)
        # x = self.dropout(x)
        x = self.classifier(x)
        # x = self.ReLU(x)
        
        y = self.backbone(y)
        # y = self.dropout(y)
        y = self.classifier(y)
        # y= self.ReLU(y)
        
        z = F.pairwise_distance(x, y, keepdim = True)
        
        return z
    

In [16]:
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []
    result = []
    pp = []
    pn = []
    nn = []
    n_p = []
    
    with torch.no_grad():
        for review_img, product_img, labels in tqdm(iter(val_loader)):
            review_img = review_img.float().to(device)
            product_img = product_img.float().to(device)

            labels = labels.type(torch.LongTensor).to(device)
            
            pred = model(review_img, product_img)
            # print(pred.shape)
            
            loss = criterion(pred, labels)
            val_loss.append(loss.item())
            
            pred = pred.detach().cpu().numpy().tolist()
            labels = labels.detach().cpu().numpy().tolist()
            
            for i in range(len(pred)):
                if labels[i] == 0:
                    if pred[i][0] < 0.5:
                        result.append(1) #맞게 예측하면 1
                        pp.append(1)
                    else:
                        result.append(0)
                        pn.append(1)
                else:
                    if pred[i][0] > 0.5:
                        result.append(1) #맞게 예측하면 1
                        nn.append(1)
                    else:
                        result.append(0)
                        n_p.append(1)
            # https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html
            
        result_len = len(result)
        print(sum(pp)/result_len)
        print(sum(pn)/result_len)
        print(sum(nn)/result_len)
        print(sum(n_p)/result_len)
        
        _val_loss = np.mean(val_loss)
        _val_score = (sum(result)/result_len) * 100 
        
        
    
    return _val_loss, _val_score

In [17]:
# Define the Contrastive Loss Function
class ContrastiveLoss(torch.nn.Module):
    def __init__(self, margin=1.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, euclidean_distance, label):
      # Calculate the euclidian distance and calculate the contrastive loss
        # loss_contrastive = torch.mean((1-label) * torch.pow(euclidean_distance, 2) +
        #                             (label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))
        # loss_contrastive = ((1-label) * torch.pow(euclidean_distance, 2) +
        #                             (label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))
        
        loss_contrastive = []
        
        for i in range(len(label)):
              loss_contrastive.append((1-label[i]) * pow(euclidean_distance[i][0], 2) +
                                    (label[i]) * pow(max(self.margin - euclidean_distance[i][0],0), 2))
        return sum(loss_contrastive)/len(label)

In [18]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model = model.to(device)
    # criterion = nn.NLLLoss(reduction="sum").to(device)
    criterion = ContrastiveLoss().to(device)
    
    best_score = 0
    best_model = None
    
    for epoch in range(0, CFG['EPOCHS']):
        model.train()
        train_loss = []
        flag_list = []
        
        for review_img, product_img, labels in tqdm(iter(train_loader)):
            
            review_img = review_img.float().to(device)
            product_img = product_img.float().to(device)
                  
            labels = labels.type(torch.LongTensor).to(device)
            
            optimizer.zero_grad()
            output = model(review_img, product_img)
            print(output)
            print(output[3][0])
            
            print(labels)
            print(labels[3])
            
            loss = criterion(output, labels)
            print(loss)
            
            # output = output.detach().cpu().numpy().tolist()
            # labels = labels.detach().cpu().numpy().tolist()
            # print(output)
            # print(output[3][0])
            # print(labels)
            # print(labels[1])
            # print(loss)
            
            # print(loss.item())
            loss.backward()
            optimizer.step()
            train_loss.append(loss.item())
                    
                
        _val_loss, _val_score = validation(model, criterion, val_loader, device)
        
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val accuracy score : [{_val_score:.5f}]')
       
        if scheduler is not None:
            scheduler.step(_val_score)
            
        if best_score < _val_score:
            best_score = _val_score
            best_model = model
            torch.save(best_model.state_dict(), "./distance_VGGBase_E_Contra.pt")
            
            
    
    return best_model

In [19]:
model = BaseModel()
model.eval()
# print(pytorch_model_summary.summary(model, torch.zeros(8,3,256,256),max_depth=None, show_parent_layers=True, show_input=True))

BaseModel(
  (backbone): EfficientNet(
    (conv_stem): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn1): BatchNormAct2d(
      32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
      (drop): Identity()
      (act): SiLU(inplace=True)
    )
    (blocks): Sequential(
      (0): Sequential(
        (0): DepthwiseSeparableConv(
          (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (bn1): BatchNormAct2d(
            32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
            (drop): Identity()
            (act): SiLU(inplace=True)
          )
          (se): SqueezeExcite(
            (conv_reduce): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (act1): SiLU(inplace=True)
            (conv_expand): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (gate): Sigmoid()
          )
          (conv_pw): Conv2d(32, 16, kernel_size=(1,

In [20]:
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2, threshold_mode='abs', min_lr=1e-8, verbose=True)

infer_model = train(model, optimizer, train_loader, val_loader, scheduler, device)

  0%|          | 0/1641 [00:03<?, ?it/s]

tensor([[2.0364],
        [1.4672],
        [0.2768],
        [0.1086],
        [0.0861],
        [0.8591],
        [0.0881],
        [0.9730]], device='cuda:0', grad_fn=<NormBackward1>)
tensor(0.1086, device='cuda:0', grad_fn=<SelectBackward0>)
tensor([1, 0, 0, 0, 1, 0, 0, 1], device='cuda:0')
tensor(0, device='cuda:0')


TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

In [ ]:
torch.save(model.state_dict(), "./distance_VGGBase_E_END.pt")